In [ ]:
%matplotlib inline
%load_ext tensorboard
#!pip install keras-tuner
import pandas as pd
import numpy as np
import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.layers import Layer
for gpu in tf.config.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(gpu, True)
from matplotlib import pyplot as plt
import keras_tuner as kt
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow.python.ops.numpy_ops import np_config

In [ ]:
genotypeFile = 'genotype.csv'
genotype = pd.read_csv(genotypeFile, sep = '\t', index_col = 0)
print('genotypeFile shape:', genotype.shape )

phenotypeFile = 'phenotype.csv'
multi_pheno = pd.read_csv(phenotypeFile, sep = ',', index_col = 0)
print('Phenotype_Multi shape:', multi_pheno.shape )


# take a small part to test code
# genotype
X = genotype
#display(X.head())
# X = genotype.iloc[0:1000:, 0:5000]
# single_pheno

#Chage the index for each model here accoding to the correspinding model
Y = multi_pheno.iloc[:, 14]#index=1 --> 1_Raffinose_1
# Y = multi_pheno.iloc[0:1000, pheno_i]


# # Add noise
# random missing masker
missing_perc = 0.1
nonmissing_ones = np.random.binomial(
    1, 1 - missing_perc, size=X.shape[0] * X.shape[1])
nonmissing_ones = nonmissing_ones.reshape(X.shape[0], X.shape[1])
nonmissing_ones, nonmissing_ones.shape

corrupted_X = X * nonmissing_ones
# corrupted_X.head()

# # Prepare data
# ## One-hot encoding

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
#X_onehot = to_categorical(X)
corrupted_X_onehot = to_categorical(corrupted_X)
# corrupted_X_onehot.shape

# normlization
scaled_Y = (Y - Y.min()) / (Y.max() - Y.min())


def detect_outliers(df):
    outlier_indices = []

    Q1 = np.percentile(df, 25)
    Q3 = np.percentile(df, 75)
    IQR = Q3 - Q1
    outlier_step = 1.5 * IQR

    outlier_indices = df[(df < Q1 - outlier_step) |
                         (df > Q3 + outlier_step)].index

    return outlier_indices


temp_Y = scaled_Y[~scaled_Y.isna()]
outliers_index = detect_outliers(temp_Y)


# set outliers as NAN
scaled_Y_ = scaled_Y.copy()
scaled_Y_[outliers_index] = np.nan


# ## Split train and test
train_X, test_X, corrupted_train_X, corrupted_test_X, train_Y, test_Y = train_test_split(
    X, corrupted_X_onehot, scaled_Y_.iloc[:], test_size=0.1)

# split df to train and valid
train_X, valid_X, corrupted_train_X, corrupted_valid_X, train_Y, valid_Y = train_test_split(
    train_X, corrupted_train_X, train_Y, test_size=0.1)



x_train = to_categorical(pd.concat([train_X,train_Y],axis=1).dropna().iloc[:,0:-1])
y_train = train_Y.dropna().to_numpy()

x_valid = to_categorical(pd.concat([valid_X,valid_Y],axis=1).dropna().iloc[:,0:-1])
y_valid = valid_Y.dropna().to_numpy()

x_test = to_categorical(pd.concat([test_X,test_Y],axis=1).dropna().iloc[:,0:-1])
y_test  = test_Y.dropna().to_numpy()

In [ ]:
x_test.shape

In [ ]:
len(x_train[:100])

In [ ]:
background_list = list([i for i in range(404)])[0:100]
if len(background_list)==0:
    background_list = x_train[np.random.choice(x_train.shape[0],100, replace=False)]
background = x_train[background_list]
background.shape

In [ ]:
import shap
from shap import initjs
initjs()
custom_objects = None
# Define custom objects if necessary
custom_objects = None

# Load the model with specified reduction strategy
model_mse = tf.keras.models.load_model('1_Raffinose_1.h5', custom_objects=custom_objects, compile=False)



In [ ]:
model_mse.summary()


In [ ]:
explainer = shap.DeepExplainer(model_mse,x_test)
base_value = explainer.expected_value

In [ ]:
base_values = base_value[0]

In [ ]:
shap_values = explainer.shap_values(x_test,check_additivity=False)

In [ ]:
mean_shap_values = np.mean(shap_values, axis=0)

In [ ]:
abs_shap_values = np.abs(shap_values[0])
feature_importance = np.mean(abs_shap_values, axis=0)
len(feature_importance)

In [ ]:
feature_importance.shape

In [ ]:
feature_impportance_sum = feature_importance.sum(axis=1)
feature_impportance_sum

In [ ]:
feature_importance_df = pd.DataFrame({
    'Feature': X.columns.tolist(),  # Convert X.columns to a list
    'Importance': feature_impportance_sum.tolist()
})

#feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
feature_importance_df

In [ ]:
num = feature_importance_df[feature_importance_df['Importance']  <= 1e-4]
less_important_features = num.Feature.tolist()
less_important_features

In [ ]:
num = feature_importance_df[feature_importance_df['Importance']  >= 1e-4]
important_features = num.Feature.tolist()
important_features

In [ ]:
def find_indexes(main_list, search_list):
    return [i for i, x in enumerate(main_list) if x in search_list]

indexes_shap = find_indexes(feature_importance_df['Feature'].tolist(), important_features)
print(len(indexes_shap))

In [ ]:
feature_importance_dict = dict(zip(feature_importance_df['Feature'], feature_importance_df['Importance']))
feature_importance_dict

In [ ]:
sorted_features = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

#### First we want to remove all values that are less than and equal to zero and save them in a list

removed_pairs_features = []

filtered_pairs_features = [(key, value) for key, value in sorted_features if value > 0]
removed_pairs_features = [(key, value) for key, value in sorted_features if value <= 0]
filtered_pairs_features_numpy = np.array(filtered_pairs_features)
# print("Filtered key-value pairs:", filtered_pairs_features)
# print("Removed key-value pairs:", removed_pairs_features)

In [ ]:
import matplotlib.pyplot as plt

# Extract feature names and importance values for visualization
sorted_feature_names = [x[0] for x in filtered_pairs_features[:20]]
sorted_feature_importance = [x[1] for x in filtered_pairs_features[:20]]

# Plot the ranked features
plt.figure(figsize=(10, 6))
plt.barh(sorted_feature_names, sorted_feature_importance)
plt.xlabel('SHAP Feature Importance')
plt.ylabel('Feature')
plt.title('Ranked Features based on SHAP Values')
plt.show()

In [ ]:
feature_shap_values = [pair[1] for pair in filtered_pairs_features[:30]]
feature_shap_values_names = [pair[0] for pair in filtered_pairs_features[:30]]
feature_names = np.array(feature_shap_values_names)
type(feature_shap_values)

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have a list of features and their corresponding SHAP values
features = feature_importance_df['Feature'][:20]
shap_values = feature_importance_df['Importance'][:20]

# Plotting
plt.figure(figsize=(8, 6))
plt.scatter(shap_values, features, color='green', s=100, alpha=0.6)
plt.xlabel('SHAP Value')
plt.ylabel('Feature')
plt.title('Feature with their corresponding SHAP value')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

In [ ]:
### Generating the FORCE Plot
shap_values = np.array(feature_shap_values)  # Assuming 100 samples and 5 features
shap_values_ = np.reshape(shap_values, (-1,2))
shap.force_plot(base_values, shap_values, feature_names)

In [ ]:
### Generating the Waterfall Plot

explanation = shap.Explanation(values=shap_values , base_values=base_values, feature_names = feature_names)

shap.waterfall_plot(explanation)

In [ ]:
genotype

In [ ]:
# Sample dataset with a categorical column
data = genotype
df_data = pd.DataFrame(data)
# Perform one-hot encoding on a specific categorical column
encoded_df = pd.get_dummies(df_data)

# Display the dataset after one-hot encoding
print("\nDataset after One-Hot Encoding:")
print(encoded_df)
encoded_df.shape

In [ ]:
import pandas as pd

# Sample one-hot encoded dataset
data = encoded_df
df_data = pd.DataFrame(data)

# Select features to set one-hot encoding to zero
selected_features = less_important_features

# Set one-hot encoding of selected features to zero
for feature in selected_features:
    for col in df_data.columns:
        if feature in col:
            df_data[col] = 0

# Check the modified dataset
print(df_data)

In [ ]:
print(df_data['5493023_chrIX_100450_T_C'])

In [ ]:
X = df_data
# X = genotype.iloc[0:1000:, 0:5000]
# single_pheno
Y = multi_pheno.iloc[:, 14]#index=1 --> 1_Raffinose_1
# Y = multi_pheno.iloc[0:1000, pheno_i]


# # Add noise
# random missing masker
missing_perc = 0.1
nonmissing_ones = np.random.binomial(
    1, 1 - missing_perc, size=X.shape[0] * X.shape[1])
nonmissing_ones = nonmissing_ones.reshape(X.shape[0], X.shape[1])
nonmissing_ones, nonmissing_ones.shape

corrupted_X = X * nonmissing_ones
print(corrupted_X['1796048_chrIV_436026_T_C'])

# # Prepare data
# ## One-hot encoding

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
#X_onehot = to_categorical(X)
corrupted_X_onehot = to_categorical(corrupted_X)
print(type(corrupted_X_onehot))
#print(corrupted_X['6810072_chrXI_231860_A_G'])
# normlization
scaled_Y = (Y - Y.min()) / (Y.max() - Y.min())


def detect_outliers(df):
    outlier_indices = []

    Q1 = np.percentile(df, 25)
    Q3 = np.percentile(df, 75)
    IQR = Q3 - Q1
    outlier_step = 1.5 * IQR

    outlier_indices = df[(df < Q1 - outlier_step) |
                         (df > Q3 + outlier_step)].index

    return outlier_indices


temp_Y = scaled_Y[~scaled_Y.isna()]
outliers_index = detect_outliers(temp_Y)


# set outliers as NAN
scaled_Y_ = scaled_Y.copy()
scaled_Y_[outliers_index] = np.nan


# ## Split train and test
train_X, test_X, corrupted_train_X, corrupted_test_X, train_Y, test_Y = train_test_split(
    X, corrupted_X_onehot, scaled_Y_.iloc[:], test_size=0.1)

# split df to train and valid
train_X, valid_X, corrupted_train_X, corrupted_valid_X, train_Y, valid_Y = train_test_split(
    train_X, corrupted_train_X, train_Y, test_size=0.1)



x_train = to_categorical(pd.concat([train_X,train_Y],axis=1).dropna().iloc[:,0:-1])
y_train = train_Y.dropna().to_numpy()

x_valid = to_categorical(pd.concat([valid_X,valid_Y],axis=1).dropna().iloc[:,0:-1])
y_valid = valid_Y.dropna().to_numpy()

x_test = to_categorical(pd.concat([test_X,test_Y],axis=1).dropna().iloc[:,0:-1])
y_test  = test_Y.dropna().to_numpy()

In [ ]:
### **Retrain the model on the new dataset**

def model_build(pheno_name,num_hl,hl_list,hl_activation, out_activation, dropout_val,filters_, kernel_size_,stride_poolSize,Conv_Layers_Stride_Size):
    

    '''
    https://towardsdatascience.com/multi-output-model-with-tensorflow-keras-functional-api-875dd89aa7c6

    def DNN_build(num_hl,hl_list,hl_activation, out_activation, dropout_val):
    pheno_name = name of phenotype of interest
    num_hl = number of hidden layers
    hl_list = list of hidden layers
    hl_activation = hidden layer activation function
    out_activation = output layer activation function
    dropout_val = Dropout value
    '''
    assert(num_hl == len(hl_list))
    assert(num_hl == len(dropout_val))
    input_layer = tf.keras.Input(shape=(28220,3))#(3138, 28220, 3)
    
    shared_convL1 = tf.keras.layers.Conv1D(filters = filters_,kernel_size = kernel_size_,strides = Conv_Layers_Stride_Size, activation = hl_activation,input_shape = (28220,3))(input_layer)
    shared_convL1_max_pool = tf.keras.layers.MaxPool1D(pool_size=stride_poolSize, strides=stride_poolSize)(shared_convL1)
    
    shared_convL2 = tf.keras.layers.Conv1D(filters = filters_,kernel_size = kernel_size_,strides = Conv_Layers_Stride_Size, activation = hl_activation)(shared_convL1_max_pool)
    shared_convL2_max_pool = tf.keras.layers.MaxPool1D(pool_size=stride_poolSize, strides=stride_poolSize)(shared_convL2)
    
    shared_convLayer_Flatten = tf.keras.layers.Flatten()(shared_convL2_max_pool)
    initializer = tf.keras.initializers.HeNormal()
    kernel_regularizer_ = tf.keras.regularizers.L1L2(l1=0.0001, l2=0.0001)
    
    shared_hidden_layer1 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[0], activation  = hl_activation)(shared_convLayer_Flatten)
    shared_hidden_layer1_dp1 = tf.keras.layers.Dropout(dropout_val[0])(shared_hidden_layer1)

    #model 1
    model1_hidden_layer2 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[1], activation  = hl_activation)(shared_hidden_layer1_dp1)
    model1_hidden_layer2_dp2 = tf.keras.layers.Dropout(dropout_val[1])(model1_hidden_layer2)

    model1_hidden_layer3 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[2], activation  = hl_activation)(model1_hidden_layer2_dp2)
    model1_hidden_layer3_dp3 = tf.keras.layers.Dropout(dropout_val[2])(model1_hidden_layer3)

    model1_hidden_layer4 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[3], activation  = hl_activation)(model1_hidden_layer3_dp3)
    model1_hidden_layer4_dp4 = tf.keras.layers.Dropout(dropout_val[3])(model1_hidden_layer4)
    model1_hidden_layer4_dp4_fl = tf.keras.layers.Flatten()(model1_hidden_layer4_dp4)
    
    pheno_name_ = tf.keras.layers.Dense(units = 1, name = pheno_name, activation  = out_activation)(model1_hidden_layer4_dp4_fl)#1_CobaltChloride_1
    model = tf.keras.models.Model(inputs = input_layer, outputs = [pheno_name_])

    return model



In [ ]:
#ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy','mse'])#change metrics to MSE
def compile_model(pheno_name,model,loss_, learningRate, metrics_): #loss = 'sparse_categorical_crossentropy'
    '''
    def compile_model(DNN, loss_, learningRate, metrics_):
    DNN: the model
    loss_: the loss function
    learningRate: learning rate
    metrics_: metrics of interest ['accuracy', 'mse']
    '''

    # Specify the optimizer, and compile the model with loss functions for both outputs
    model.compile(
       #optimizer = tf.keras.optimizers.Adam(learning_rate=learningRate),#0.00027705 
    optimizer =  tf.keras.optimizers.legacy.SGD(learning_rate=learningRate, momentum=0.01, nesterov=True, name="SGD"),
      loss = {
          pheno_name:loss_
      }, 
      metrics = [metrics_]#{'CobaltChloride_1':[metrics_],'CopperSulfate_1':[metrics_],'Diamide_1':[metrics_]}
      )#change metrics to MSE ##['accuracy','mse']
    return model

def runModel(model, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode,pheno_index):
    '''
    def buildModel(DNN, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode):
    DNN: DNN which the model
    val_split_size: the validation split)
    batch_size_: batch_size
    numEpochs: number of epochs
    patience_: patience of call back
    monitor_: monitor (objective of callback)
    mode: mode (min, max, auto)
    pheno_index: the index of the phenotype in y_train
    '''
    history = model.fit(
    x_train, 
    [y_train[:]],
    validation_data=(x_valid, [y_valid[:]]),
    batch_size = batch_size_, 
    epochs = numEpochs,
    callbacks = [
      tf.keras.callbacks.EarlyStopping(monitor= monitor_,patience=patience_,verbose=1,mode=mode),#monitoring loss mode should be min [---val_acc--]
      #tf.keras.callbacks.ModelCheckpoint(filepath='./TrainedModels/model.{epoch:02d}-{val_loss:.2f}.h5',save_best_only=True),
      #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    ]
    )

    return history



In [ ]:
def runModel_all(pheno_name, pheno_index,hl_list, drop_list):
    model_mse = model_build(pheno_name,4,hl_list,'relu','linear',drop_list,8,7,2,3)#model initiallization
    model_mse = compile_model(pheno_name,model_mse,tf.keras.losses.MeanSquaredError(reduction="auto", name="mse"),
    0.00123695244,'mse')
    history_mse =  runModel(model_mse,0.30,32,400,15, "val_mse", "min",pheno_index)
    return history_mse,model_mse

In [ ]:
history_mse,model_mse = runModel_all('1_Raffinose_1',14,[1464,608,264,146],[0.00,0.00,0.00,0.20])

In [ ]:
model_mse.save('important features/saved Models/1_Raffinose_1.h5')

In [ ]:
from sklearn.metrics import mean_squared_error

# Make predictions on the validation set
y_pred = model_mse.predict(x_valid)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_valid, y_pred)
print("Mean Squared Error on the validation set:", mse)

# Make predictions on the tesing set
y_pred = model_mse.predict(x_test)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on the testing set:", mse)


In [ ]:
#### This is for LIME
import lime
import lime.lime_tabular
model_mse= tf.keras.models.load_model('1_Raffinose_1.h5')
print(type(genotype))
genotype_data = genotype.to_numpy()
class_names = ['Positive','Negative']
my_features = genotype.columns.tolist()
#model_mse_ = tf.keras.models.load_model('important features/Models/1_CobaltChloride_1.h5', custom_objects=custom_objects, compile=False)

explainer = lime.lime_tabular.RecurrentTabularExplainer(x_train, mode='regression', training_labels=None, feature_names=my_features, 
                        categorical_features=None, categorical_names=None, kernel_width=None, kernel=None, verbose=False, 
                        class_names=class_names, feature_selection='auto', discretize_continuous=True, discretizer='quartile', 
                         random_state=None)

In [ ]:
i = 0
instance = x_test[i]
exp = explainer.explain_instance(instance, model_mse.predict, num_features = len(my_features))

In [ ]:
exp.as_list()
lime_importance = dict(exp.as_list())

In [ ]:
sorted_lime_importance = dict(sorted(lime_importance.items(), key=lambda item: item[1], reverse=True))
sorted_lime_importance

In [ ]:
less_important_lime = [k for k, v in sorted_lime_importance.items() if v <= 1e-5]
len(less_important_lime)

In [ ]:
important_lime = [k for k, v in lime_importance.items() if v >= 1e-5]
len(important_lime)

In [ ]:
def find_indexes(main_list, search_list):
    return [i for i, x in enumerate(main_list) if x in search_list]

indexes_lime = find_indexes(lime_importance, important_lime)
len(indexes_lime)

In [ ]:
def find_indexes(main_list, search_list):
    return [i for i, x in enumerate(main_list) if x in search_list]

indexes = find_indexes(lime_importance, less_important_lime)
print(indexes) 

In [ ]:
import matplotlib.pyplot as plt

sorted_lime_importance_list = sorted(sorted_lime_importance.items(), key=lambda item: item[1], reverse=True)

num_features_to_plot = min(20, len(sorted_lime_importance_list))

# Extract feature names and importance values for visualization
sorted_lime_names = [x[0] for x in sorted_lime_importance_list[:num_features_to_plot]]
sorted_lime_importance_ = [x[1] for x in sorted_lime_importance_list[:num_features_to_plot]]

# Plot the ranked features
plt.figure(figsize=(10, 6))
plt.barh(sorted_lime_names, sorted_lime_importance_)
plt.xlabel('LIME Feature Importance')
plt.ylabel('Feature')
plt.title('Ranked Features based on LIME Values')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

sorted_lime_importance_list = sorted(sorted_lime_importance.items(), key=lambda item: item[1], reverse=True)

num_features_to_plot = min(20, len(sorted_lime_importance_list))

# Extract feature names and importance values for visualization
sorted_lime_names = [x[0] for x in sorted_lime_importance_list[:num_features_to_plot]]
sorted_lime_importance_ = [x[1] for x in sorted_lime_importance_list[:num_features_to_plot]]
# Plotting
plt.figure(figsize=(8, 6))
plt.scatter(sorted_lime_importance_, sorted_lime_names, color='green', s=100, alpha=0.6)
plt.xlabel('LIME Value')
plt.ylabel('Feature')
plt.title('Feature with their corresponding SHAP value')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

In [ ]:
# Sample dataset with a categorical column
data_lime = genotype
df_data = pd.DataFrame(data_lime)
# Perform one-hot encoding on a specific categorical column
encoded_df = pd.get_dummies(df_data)

# Display the dataset after one-hot encoding
print("\nDataset after One-Hot Encoding:")
print(encoded_df)
encoded_df.shape

In [ ]:
import pandas as pd

# Sample one-hot encoded dataset
data = encoded_df
df_data_ = pd.DataFrame(data)
type(df_data.columns)
# Select features to set one-hot encoding to zero
selected_features = indexes

# Set one-hot encoding of selected features to zero
for index in selected_features:
    for col in df_data_.columns:
        if index in df_data_.columns:
            df_data_[col] = 0

# Check the modified dataset
print(df_data_)

In [ ]:
X = df_data_
# X = genotype.iloc[0:1000:, 0:5000]
# single_pheno
Y = multi_pheno.iloc[:, 14]#index=1 --> 1_Raffinose_1
# Y = multi_pheno.iloc[0:1000, pheno_i]


# # Add noise
# random missing masker
missing_perc = 0.1
nonmissing_ones = np.random.binomial(
    1, 1 - missing_perc, size=X.shape[0] * X.shape[1])
nonmissing_ones = nonmissing_ones.reshape(X.shape[0], X.shape[1])
nonmissing_ones, nonmissing_ones.shape

corrupted_X = X * nonmissing_ones

# # Prepare data
# ## One-hot encoding

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
#X_onehot = to_categorical(X)
corrupted_X_onehot = to_categorical(corrupted_X)
print(type(corrupted_X_onehot))
#print(corrupted_X['6810072_chrXI_231860_A_G'])
# normlization
scaled_Y = (Y - Y.min()) / (Y.max() - Y.min())


def detect_outliers(df):
    outlier_indices = []

    Q1 = np.percentile(df, 25)
    Q3 = np.percentile(df, 75)
    IQR = Q3 - Q1
    outlier_step = 1.5 * IQR

    outlier_indices = df[(df < Q1 - outlier_step) |
                         (df > Q3 + outlier_step)].index

    return outlier_indices


temp_Y = scaled_Y[~scaled_Y.isna()]
outliers_index = detect_outliers(temp_Y)


# set outliers as NAN
scaled_Y_ = scaled_Y.copy()
scaled_Y_[outliers_index] = np.nan


# ## Split train and test
train_X, test_X, corrupted_train_X, corrupted_test_X, train_Y, test_Y = train_test_split(
    X, corrupted_X_onehot, scaled_Y_.iloc[:], test_size=0.1)

# split df to train and valid
train_X, valid_X, corrupted_train_X, corrupted_valid_X, train_Y, valid_Y = train_test_split(
    train_X, corrupted_train_X, train_Y, test_size=0.1)



x_train = to_categorical(pd.concat([train_X,train_Y],axis=1).dropna().iloc[:,0:-1])
y_train = train_Y.dropna().to_numpy()

x_valid = to_categorical(pd.concat([valid_X,valid_Y],axis=1).dropna().iloc[:,0:-1])
y_valid = valid_Y.dropna().to_numpy()

x_test = to_categorical(pd.concat([test_X,test_Y],axis=1).dropna().iloc[:,0:-1])
y_test  = test_Y.dropna().to_numpy()

In [ ]:
### **Retrain the model on the new dataset**

def model_build(pheno_name,num_hl,hl_list,hl_activation, out_activation, dropout_val,filters_, kernel_size_,stride_poolSize,Conv_Layers_Stride_Size):
    

    '''
    https://towardsdatascience.com/multi-output-model-with-tensorflow-keras-functional-api-875dd89aa7c6

    def DNN_build(num_hl,hl_list,hl_activation, out_activation, dropout_val):
    pheno_name = name of phenotype of interest
    num_hl = number of hidden layers
    hl_list = list of hidden layers
    hl_activation = hidden layer activation function
    out_activation = output layer activation function
    dropout_val = Dropout value
    '''
    assert(num_hl == len(hl_list))
    assert(num_hl == len(dropout_val))
    input_layer = tf.keras.Input(shape=(28220,3))#(3138, 28220, 3)
    
    shared_convL1 = tf.keras.layers.Conv1D(filters = filters_,kernel_size = kernel_size_,strides = Conv_Layers_Stride_Size, activation = hl_activation,input_shape = (28220,3))(input_layer)
    shared_convL1_max_pool = tf.keras.layers.MaxPool1D(pool_size=stride_poolSize, strides=stride_poolSize)(shared_convL1)
    
    shared_convL2 = tf.keras.layers.Conv1D(filters = filters_,kernel_size = kernel_size_,strides = Conv_Layers_Stride_Size, activation = hl_activation)(shared_convL1_max_pool)
    shared_convL2_max_pool = tf.keras.layers.MaxPool1D(pool_size=stride_poolSize, strides=stride_poolSize)(shared_convL2)
    
    shared_convLayer_Flatten = tf.keras.layers.Flatten()(shared_convL2_max_pool)
    initializer = tf.keras.initializers.HeNormal()
    kernel_regularizer_ = tf.keras.regularizers.L1L2(l1=0.0001, l2=0.0001)
    
    shared_hidden_layer1 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[0], activation  = hl_activation)(shared_convLayer_Flatten)
    shared_hidden_layer1_dp1 = tf.keras.layers.Dropout(dropout_val[0])(shared_hidden_layer1)

    #model 1
    model1_hidden_layer2 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[1], activation  = hl_activation)(shared_hidden_layer1_dp1)
    model1_hidden_layer2_dp2 = tf.keras.layers.Dropout(dropout_val[1])(model1_hidden_layer2)

    model1_hidden_layer3 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[2], activation  = hl_activation)(model1_hidden_layer2_dp2)
    model1_hidden_layer3_dp3 = tf.keras.layers.Dropout(dropout_val[2])(model1_hidden_layer3)

    model1_hidden_layer4 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[3], activation  = hl_activation)(model1_hidden_layer3_dp3)
    model1_hidden_layer4_dp4 = tf.keras.layers.Dropout(dropout_val[3])(model1_hidden_layer4)
    model1_hidden_layer4_dp4_fl = tf.keras.layers.Flatten()(model1_hidden_layer4_dp4)
    
    pheno_name_ = tf.keras.layers.Dense(units = 1, name = pheno_name, activation  = out_activation)(model1_hidden_layer4_dp4_fl)#1_CobaltChloride_1
    model = tf.keras.models.Model(inputs = input_layer, outputs = [pheno_name_])

    return model



In [ ]:
#ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy','mse'])#change metrics to MSE
def compile_model(pheno_name,model,loss_, learningRate, metrics_): #loss = 'sparse_categorical_crossentropy'
    '''
    def compile_model(DNN, loss_, learningRate, metrics_):
    DNN: the model
    loss_: the loss function
    learningRate: learning rate
    metrics_: metrics of interest ['accuracy', 'mse']
    '''

    # Specify the optimizer, and compile the model with loss functions for both outputs
    model.compile(
       #optimizer = tf.keras.optimizers.Adam(learning_rate=learningRate),#0.00027705 
    optimizer =  tf.keras.optimizers.legacy.SGD(learning_rate=learningRate, momentum=0.01, nesterov=True, name="SGD"),
      loss = {
          pheno_name:loss_
      }, 
      metrics = [metrics_]#{'CobaltChloride_1':[metrics_],'CopperSulfate_1':[metrics_],'Diamide_1':[metrics_]}
      )#change metrics to MSE ##['accuracy','mse']
    return model

def runModel(model, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode,pheno_index):
    '''
    def buildModel(DNN, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode):
    DNN: DNN which the model
    val_split_size: the validation split)
    batch_size_: batch_size
    numEpochs: number of epochs
    patience_: patience of call back
    monitor_: monitor (objective of callback)
    mode: mode (min, max, auto)
    pheno_index: the index of the phenotype in y_train
    '''
    history = model.fit(
    x_train, 
    [y_train[:]],
    validation_data=(x_valid, [y_valid[:]]),
    batch_size = batch_size_, 
    epochs = numEpochs,
    callbacks = [
      tf.keras.callbacks.EarlyStopping(monitor= monitor_,patience=patience_,verbose=1,mode=mode),#monitoring loss mode should be min [---val_acc--]
      #tf.keras.callbacks.ModelCheckpoint(filepath='./TrainedModels/model.{epoch:02d}-{val_loss:.2f}.h5',save_best_only=True),
      #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    ]
    )

    return history



In [ ]:
def runModel_all(pheno_name, pheno_index,hl_list, drop_list):
    model_mse = model_build(pheno_name,4,hl_list,'relu','linear',drop_list,8,7,2,3)#model initiallization
    model_mse = compile_model(pheno_name,model_mse,tf.keras.losses.MeanSquaredError(reduction="auto", name="mse"),
    0.00123695244,'mse')
    history_mse =  runModel(model_mse,0.30,32,400,15, "val_mse", "min",pheno_index)
    return history_mse,model_mse

In [ ]:
history_mse,model_mse = runModel_all('1_Raffinose_1',14,[1464,608,264,146],[0.00,0.00,0.00,0.20])

In [ ]:
from sklearn.metrics import mean_squared_error

# Make predictions on the validation set
y_pred = model_mse.predict(x_valid)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_valid, y_pred)
print("Mean Squared Error on the validation set:", mse)

# Make predictions on the tesing set
y_pred = model_mse.predict(x_test)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on the testing set:", mse)


In [ ]:
###### Intersection of LIME and SHAP 

In [ ]:
def find_common_indexes(lime_indexes, shap_indexes):
    # Convert lists to sets
    lime_set = set(lime_indexes)
    shap_set = set(shap_indexes)
    
    # Find the intersection
    common_indexes = lime_set.intersection(shap_set)
    
    return common_indexes
    

common_indexes = find_common_indexes(indexes_lime, indexes_shap)
print(type(common_indexes))


In [ ]:
common_indexes = np.array(list(common_indexes))
(common_indexes)

In [ ]:
common_indexes_ = common_indexes.tolist()
len(common_indexes_)

In [ ]:
def get_strings_at_indices(strings, indices):

    valid_indices = [index for index in indices if 0 <= index < len(strings)]
    
    # Get strings at valid indices
    result = [strings[index] for index in valid_indices]
    
    return result

matching_features = get_strings_at_indices(feature_importance_df['Feature'].tolist(), common_indexes_)
len(matching_features)

In [ ]:
genotypeFile = 'genotype.csv'
genotype = pd.read_csv(genotypeFile, sep = '\t', index_col = 0)
print('genotypeFile shape:', genotype.shape )

phenotypeFile = 'phenotype.csv'
multi_pheno = pd.read_csv(phenotypeFile, sep = ',', index_col = 0)
print('Phenotype_Multi shape:', multi_pheno.shape )


# take a small part to test code
# genotype
X = genotype

mask = ~X.columns.str.contains('|'.join(matching_features))

# Set values to zero for non-matching columns
X.loc[:, mask] = 0

#display(X.head())
# X = genotype.iloc[0:1000:, 0:5000]
# single_pheno

#Chage the index for each model here accoding to the correspinding model
Y = multi_pheno.iloc[:, 14]#index=0 --> 1_Raffinose_1
# # Y = multi_pheno.iloc[0:1000, pheno_i]


# # Add noise
# random missing masker
missing_perc = 0.1
nonmissing_ones = np.random.binomial(
    1, 1 - missing_perc, size=X.shape[0] * X.shape[1])
nonmissing_ones = nonmissing_ones.reshape(X.shape[0], X.shape[1])
nonmissing_ones, nonmissing_ones.shape

corrupted_X = X * nonmissing_ones
# corrupted_X.head()

# # Prepare data
# ## One-hot encoding

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
#X_onehot = to_categorical(X)
corrupted_X_onehot = to_categorical(corrupted_X)
# corrupted_X_onehot.shape

# normlization
scaled_Y = (Y - Y.min()) / (Y.max() - Y.min())


def detect_outliers(df):
    outlier_indices = []

    Q1 = np.percentile(df, 25)
    Q3 = np.percentile(df, 75)
    IQR = Q3 - Q1
    outlier_step = 1.5 * IQR

    outlier_indices = df[(df < Q1 - outlier_step) |
                         (df > Q3 + outlier_step)].index

    return outlier_indices


temp_Y = scaled_Y[~scaled_Y.isna()]
outliers_index = detect_outliers(temp_Y)


# set outliers as NAN
scaled_Y_ = scaled_Y.copy()
scaled_Y_[outliers_index] = np.nan


# ## Split train and test
train_X, test_X, corrupted_train_X, corrupted_test_X, train_Y, test_Y = train_test_split(
    X, corrupted_X_onehot, scaled_Y_.iloc[:], test_size=0.1)

# split df to train and valid
train_X, valid_X, corrupted_train_X, corrupted_valid_X, train_Y, valid_Y = train_test_split(
    train_X, corrupted_train_X, train_Y, test_size=0.1)



x_train = to_categorical(pd.concat([train_X,train_Y],axis=1).dropna().iloc[:,0:-1])
y_train = train_Y.dropna().to_numpy()

x_valid = to_categorical(pd.concat([valid_X,valid_Y],axis=1).dropna().iloc[:,0:-1])
y_valid = valid_Y.dropna().to_numpy()

x_test = to_categorical(pd.concat([test_X,test_Y],axis=1).dropna().iloc[:,0:-1])
y_test  = test_Y.dropna().to_numpy()

In [ ]:
### **Retrain the model on the new dataset**

def model_build(pheno_name,num_hl,hl_list,hl_activation, out_activation, dropout_val,filters_, kernel_size_,stride_poolSize,Conv_Layers_Stride_Size):
    

    '''
    https://towardsdatascience.com/multi-output-model-with-tensorflow-keras-functional-api-875dd89aa7c6

    def DNN_build(num_hl,hl_list,hl_activation, out_activation, dropout_val):
    pheno_name = name of phenotype of interest
    num_hl = number of hidden layers
    hl_list = list of hidden layers
    hl_activation = hidden layer activation function
    out_activation = output layer activation function
    dropout_val = Dropout value
    '''
    assert(num_hl == len(hl_list))
    assert(num_hl == len(dropout_val))
    input_layer = tf.keras.Input(shape=(28220,3))#(3138, 28220, 3)
    
    shared_convL1 = tf.keras.layers.Conv1D(filters = filters_,kernel_size = kernel_size_,strides = Conv_Layers_Stride_Size, activation = hl_activation,input_shape = (28220,3))(input_layer)
    shared_convL1_max_pool = tf.keras.layers.MaxPool1D(pool_size=stride_poolSize, strides=stride_poolSize)(shared_convL1)
    
    shared_convL2 = tf.keras.layers.Conv1D(filters = filters_,kernel_size = kernel_size_,strides = Conv_Layers_Stride_Size, activation = hl_activation)(shared_convL1_max_pool)
    shared_convL2_max_pool = tf.keras.layers.MaxPool1D(pool_size=stride_poolSize, strides=stride_poolSize)(shared_convL2)
    
    shared_convLayer_Flatten = tf.keras.layers.Flatten()(shared_convL2_max_pool)
    initializer = tf.keras.initializers.HeNormal()
    kernel_regularizer_ = tf.keras.regularizers.L1L2(l1=0.0001, l2=0.0001)
    
    shared_hidden_layer1 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[0], activation  = hl_activation)(shared_convLayer_Flatten)
    shared_hidden_layer1_dp1 = tf.keras.layers.Dropout(dropout_val[0])(shared_hidden_layer1)

    #model 1
    model1_hidden_layer2 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[1], activation  = hl_activation)(shared_hidden_layer1_dp1)
    model1_hidden_layer2_dp2 = tf.keras.layers.Dropout(dropout_val[1])(model1_hidden_layer2)

    model1_hidden_layer3 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[2], activation  = hl_activation)(model1_hidden_layer2_dp2)
    model1_hidden_layer3_dp3 = tf.keras.layers.Dropout(dropout_val[2])(model1_hidden_layer3)

    model1_hidden_layer4 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[3], activation  = hl_activation)(model1_hidden_layer3_dp3)
    model1_hidden_layer4_dp4 = tf.keras.layers.Dropout(dropout_val[3])(model1_hidden_layer4)
    model1_hidden_layer4_dp4_fl = tf.keras.layers.Flatten()(model1_hidden_layer4_dp4)
    
    pheno_name_ = tf.keras.layers.Dense(units = 1, name = pheno_name, activation  = out_activation)(model1_hidden_layer4_dp4_fl)#1_CobaltChloride_1
    model = tf.keras.models.Model(inputs = input_layer, outputs = [pheno_name_])

    return model



In [ ]:
#ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy','mse'])#change metrics to MSE
def compile_model(pheno_name,model,loss_, learningRate, metrics_): #loss = 'sparse_categorical_crossentropy'
    '''
    def compile_model(DNN, loss_, learningRate, metrics_):
    DNN: the model
    loss_: the loss function
    learningRate: learning rate
    metrics_: metrics of interest ['accuracy', 'mse']
    '''

    # Specify the optimizer, and compile the model with loss functions for both outputs
    model.compile(
       #optimizer = tf.keras.optimizers.Adam(learning_rate=learningRate),#0.00027705 
    optimizer =  tf.keras.optimizers.legacy.SGD(learning_rate=learningRate, momentum=0.01, nesterov=True, name="SGD"),
      loss = {
          pheno_name:loss_
      }, 
      metrics = [metrics_]#{'CobaltChloride_1':[metrics_],'CopperSulfate_1':[metrics_],'Diamide_1':[metrics_]}
      )#change metrics to MSE ##['accuracy','mse']
    return model

def runModel(model, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode,pheno_index):
    '''
    def buildModel(DNN, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode):
    DNN: DNN which the model
    val_split_size: the validation split)
    batch_size_: batch_size
    numEpochs: number of epochs
    patience_: patience of call back
    monitor_: monitor (objective of callback)
    mode: mode (min, max, auto)
    pheno_index: the index of the phenotype in y_train
    '''
    history = model.fit(
    x_train, 
    [y_train[:]],
    validation_data=(x_valid, [y_valid[:]]),
    batch_size = batch_size_, 
    epochs = numEpochs,
    callbacks = [
      tf.keras.callbacks.EarlyStopping(monitor= monitor_,patience=patience_,verbose=1,mode=mode),#monitoring loss mode should be min [---val_acc--]
      #tf.keras.callbacks.ModelCheckpoint(filepath='./TrainedModels/model.{epoch:02d}-{val_loss:.2f}.h5',save_best_only=True),
      #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    ]
    )

    return history



In [ ]:
def runModel_all(pheno_name, pheno_index,hl_list, drop_list):
    model_mse = model_build(pheno_name,4,hl_list,'relu','linear',drop_list,8,7,2,3)#model initiallization
    model_mse = compile_model(pheno_name,model_mse,tf.keras.losses.MeanSquaredError(reduction="auto", name="mse"),
    0.00123695244,'mse')
    history_mse =  runModel(model_mse,0.30,32,400,15, "val_mse", "min",pheno_index)
    return history_mse,model_mse

In [ ]:
history_mse,model_mse = runModel_all('1_Raffinose_1',14,[1464,608,264,146],[0.00,0.00,0.00,0.20])

In [ ]:
from sklearn.metrics import mean_squared_error

# Make predictions on the validation set
y_pred = model_mse.predict(x_valid)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_valid, y_pred)
print("Mean Squared Error on the validation set:", mse)

# Make predictions on the tesing set
y_pred = model_mse.predict(x_test)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on the testing set:", mse)
